In [150]:
import pandas as pd
import numpy as np
import random

In [292]:
df = pd.read_csv('Purchase_Card_Transactions.csv')
df['MCC_DESCRIPTION'] = df['MCC_DESCRIPTION'].apply(lambda x: str(x).replace('-', ' ').upper())

df_group = df.groupby(['AGENCY', 'MCC_DESCRIPTION'])['TRANSACTION_AMOUNT'].agg(['sum', 'count'])
df_numpy = df_group.to_numpy()

agency_spending = df.groupby(['AGENCY'])['TRANSACTION_AMOUNT'].agg(['sum']).reset_index()
vendor_income = df.groupby(['MCC_DESCRIPTION'])['TRANSACTION_AMOUNT'].agg(['sum']).reset_index()

hash_a = {}
list_a = []
a_val = 0
hash_v = {}
v_val = 0
list_v = []
ret_list = []

for i in range(len(df_numpy)):
    a_name = df_group.iloc[i].name[0]
    v_name = df_group.iloc[i].name[1]
    if a_name not in hash_a:
        hash_a[a_name] = a_val
        a_val += 1
        list_a.append(a_name)
    if v_name not in hash_v:
        hash_v[v_name] = v_val
        v_val += 1
        list_v.append(v_name)
    s = df_numpy[i][0]
    #print(agency_spending[agency_spending["AGENCY"]==a_name].reset_index()['sum'][0])
    ret_list.append([hash_a[a_name], hash_v[v_name], abs(s)/agency_spending[agency_spending["AGENCY"]==a_name].reset_index()['sum'][0]])
    #ret_list.append([hash_a[a_name], hash_v[v_name], 1])

print("Done!")

Done!


In [185]:
df = pd.read_csv('Purchase_Card_Transactions.csv')
#df['MCC_DESCRIPTION'] = df['MCC_DESCRIPTION'].apply(lambda x: str(x).replace('-', ' ').upper())

df_group = df.groupby(['VENDOR_NAME', 'AGENCY'])['TRANSACTION_AMOUNT'].agg(['sum', 'count'])
df_numpy = df_group.to_numpy()

agency_spending = df.groupby(['AGENCY'])['TRANSACTION_AMOUNT'].agg(['sum']).reset_index()

hash_a = {}
list_a = []
a_val = 0
hash_v = {}
v_val = 0
list_v = []
ret_list = []

for i in range(round(len(df_numpy)/40)):
    a_name = df_group.iloc[i].name[1]
    v_name = df_group.iloc[i].name[0]
    if a_name not in hash_a:
        hash_a[a_name] = a_val
        a_val += 1
        list_a.append(a_name)
    if v_name not in hash_v:
        hash_v[v_name] = v_val
        v_val += 1
        list_v.append(v_name)
    s = df_numpy[i][0]
    #print(agency_spending[agency_spending["AGENCY"]==a_name].reset_index()['sum'][0])
    ret_list.append([hash_a[a_name], hash_v[v_name], abs(s)/agency_spending[agency_spending["AGENCY"]==a_name].reset_index()['sum'][0]])
    #ret_list.append([hash_a[a_name], hash_v[v_name], 1])

print("Done!")

                                         AGENCY         sum
0              Advisory Neighborhood Commission    32574.30
1  Alcoholic Beverage Regulation Administration   918991.51
2                   Board of Elections & Ethics  1317784.88
3   Board of Real Property Assessment & Appeals   640783.72
4        Child & Family Services Administration  5117774.31
Done!


In [285]:
def build_clusters(set_grouping, n_nodes, edge_matrix, n_clusters, boolClusterAgency):
    #bool cluster agency indicates whether agencies are being clustered (1) or vendors (0)

    
    rho = .0003
    new_clusters = np.arange(n_nodes)
    new_cluster_groupings = [[i] for i in range(n_nodes)]

    cluster_weights = np.zeros([len(set_grouping),n_nodes])
    for cluster_num in range(len(set_grouping)):
        for part in set_grouping[cluster_num]:
            node_indices = np.where(edge_matrix[:,boolClusterAgency] == part)
            for edge_idx in node_indices[0]:
                cluster_weights[cluster_num,int(edge_matrix[edge_idx,1-boolClusterAgency])] += edge_matrix[edge_idx,2]/len(set_grouping[cluster_num])
                #TODO: sum of cluster weights?  max?  average?
                
    #print(cluster_weights)

    node_similarity_matrix = 10000*np.ones([n_nodes, n_nodes])
    for node1 in range(n_nodes):
        #for node2 in range(node1 + 1,n_nodes):
        for node2 in range(node1+1,n_nodes):
            node_similarity_matrix[node1,node2] = np.mean(np.abs((cluster_weights[:,node1] - cluster_weights[:,node2])))
            node_similarity_matrix[node2,node1] = np.mean(np.abs((cluster_weights[:,node1] - cluster_weights[:,node2])))
        #if node1 % 100 == 0:
        #    print(str(node1) + "/" + str(n_nodes))\
        
    #print(node_similarity_matrix)
        
            
    print("Processing Done")
    
    total_clusters = n_nodes

    while True:

        (node1, node2) = np.unravel_index(np.argmin(node_similarity_matrix), node_similarity_matrix.shape)
        rm_cluster = new_clusters[node2]
        add_cluster = new_clusters[node1]

        #for rm_node in new_cluster_groupings[rm_cluster]:
        #    new_clusters[rm_node] = add_cluster
        #    for ad_node in new_cluster_groupings[add_cluster]:
        #        node_similarity_matrix[ad_node, rm_node] = 0
        #        node_similarity_matrix[rm_node, ad_node] = 0
        
        
        
        node_similarity_matrix[node1,:] = (len(new_cluster_groupings[rm_cluster])*node_similarity_matrix[node2,:] + len(new_cluster_groupings[add_cluster])*node_similarity_matrix[node1,:])/(len(new_cluster_groupings[add_cluster]) + len(new_cluster_groupings[rm_cluster])) + rho*((len(new_cluster_groupings[add_cluster]) + len(new_cluster_groupings[rm_cluster]))*n_clusters/n_nodes)**2
        node_similarity_matrix[:,node1] = (len(new_cluster_groupings[rm_cluster])*node_similarity_matrix[:,node2] + len(new_cluster_groupings[add_cluster])*node_similarity_matrix[:,node1])/(len(new_cluster_groupings[add_cluster]) + len(new_cluster_groupings[rm_cluster])) + rho*((len(new_cluster_groupings[add_cluster]) + len(new_cluster_groupings[rm_cluster]))*n_clusters/n_nodes)**2
        
        node_similarity_matrix[node2,:] = 10000
        node_similarity_matrix[:,node2] = 10000
        
        
        for rm_node in new_cluster_groupings[rm_cluster]:
            new_clusters[rm_node] = add_cluster
            
            for ad_node in new_cluster_groupings[add_cluster]:
                node_similarity_matrix[ad_node, rm_node] = 10000
                node_similarity_matrix[rm_node, ad_node] = 10000
        
        #for rm_node in new_cluster_groupings[rm_cluster]:
        #    new_clusters[rm_node] = add_cluster

        new_cluster_groupings[add_cluster].extend(new_cluster_groupings[rm_cluster])
        new_cluster_groupings[rm_cluster] = []

        
        #TODO: group based on nearest node?  nearest cluster?

        total_clusters -= 1

        if n_clusters >= total_clusters:

            """final_cluster_groupings = []
            for i in range(len(new_cluster_groupings)):
                if len(new_cluster_groupings[i]) > 0:
                    final_cluster_groupings.append(new_cluster_groupings[i])
                    for n in new_cluster_groupings[i]:
                        new_clusters[n] = len(final_cluster_groupings) - 1"""

            #print(np.where(node_similarity_matrix != 0))
            
            final_clusters, final_cluster_groupings = build_cluster_sets_vector(new_clusters)
            
            return final_clusters, final_cluster_groupings

In [286]:
def build_cluster_sets_vector(set_ids):
    
    unique_sets = np.unique(set_ids)
    cluster_sets = [[] for i in range(len(unique_sets))]
    new_set_ids = np.zeros(len(set_ids),)
    for i in range(len(unique_sets)):
        indices = np.where(set_ids==unique_sets[i])
        cluster_sets[i].extend(indices[0])
        new_set_ids[indices] = i
        
        
        
    return new_set_ids, cluster_sets
        
    

In [287]:
def cluster(n_agency, n_vendors, edge_matrix, iter, init_vendor_cluster = None):
    #n_agency - number of agencies
    #n_vendors - number of vendors
    #edge_matrix - matrix of [agency node index, vendor node index, edge weight]
    #iter - number of clustering iterations to run
    #init_vendor_cluster


    vendor_cluster_groupings = init_vendor_cluster
    if init_vendor_cluster == None:
        vendor_cluster_groupings = [[i] for i in range(n_vendors)]

    for i in range(iter):
        print(i)

        agency_clusters, agency_cluster_groupings = build_clusters(vendor_cluster_groupings, n_agency, edge_matrix, 15, 1)

        #print(agency_clusters)
        #print(agency_cluster_groupings)
        vendor_clusters, vendor_cluster_groupings = build_clusters(agency_cluster_groupings, n_vendors, edge_matrix, 100, 0)
        #print(vendor_clusters)
        #print(vendor_cluster_groupings)
    return agency_clusters, agency_cluster_groupings, vendor_clusters, vendor_cluster_groupings

In [289]:
ag_cluster, ag_groups, v_cluster, v_groups = cluster(len(list_a), len(list_v), np.array(ret_list), 1, init_vendor_cluster = None)

0
Processing Done
Processing Done


In [ ]:
print(np.array(ret_list))

In [ ]:
x = df.groupby(['AGENCY'])['TRANSACTION_AMOUNT'].agg(['sum']).reset_index()

In [ ]:
x[x["AGENCY"]=="Advisory Neighborhood Commission"]['sum'][0]

In [ ]:
node_similarity_matrix.argmax(axis=1)

In [290]:
v_groups

[[0, 223],
 [1, 527],
 [2, 541],
 [3, 557],
 [4, 561],
 [5, 6, 9, 14, 20, 23, 35, 41, 46, 48, 77, 90, 110, 148],
 [7, 51, 58, 94, 95, 103, 111, 113, 133, 179, 187, 254, 308, 317, 344, 376],
 [8, 37, 47, 60, 62, 87, 96, 105, 130, 160, 324, 330, 401, 413],
 [10, 44, 71, 80, 84, 88, 91, 104, 145, 146, 181, 211, 226, 237],
 [11, 25],
 [12,
  13,
  33,
  36,
  69,
  75,
  79,
  85,
  101,
  112,
  143,
  149,
  151,
  189,
  217,
  221,
  250,
  279,
  297],
 [15, 157],
 [16, 19, 45, 49, 52, 54, 100, 107, 114, 135, 144, 185, 269, 402, 407],
 [17,
  61,
  129,
  172,
  193,
  201,
  240,
  256,
  264,
  325,
  345,
  354,
  415,
  452,
  461,
  487],
 [18, 32, 38, 43, 65, 70, 78, 109, 126, 204, 271, 284, 294],
 [21, 298, 323, 378, 408],
 [22],
 [24],
 [26],
 [27, 73, 97, 108, 128, 167],
 [28,
  30,
  31,
  86,
  156,
  163,
  202,
  268,
  333,
  434,
  470,
  471,
  475,
  485,
  523,
  554,
  558,
  602,
  603,
  604,
  605,
  606],
 [29, 288],
 [34, 118, 410],
 [39],
 [40,
  50,
  53,
  5

In [278]:
indices = [0, 56]

for i in indices:
    print(list_a[i])

print("\n")
indices2 = [212, 228]
for j in indices2:
    print(list_v[j])

Advisory Neighborhood Commission
Office of Community Affairs


HOLIDAY INNS
LODGING HOTELS MOTELS RESORTS NOT CLASSIFIED


In [291]:
ag_groups

[[0, 4],
 [1, 32],
 [2, 35],
 [3, 38],
 [5,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  19,
  20,
  21,
  23,
  27,
  30,
  31,
  47,
  48,
  52,
  53,
  54,
  55,
  58,
  59,
  60,
  65,
  67,
  68,
  69,
  70,
  71,
  72,
  74,
  76,
  77,
  84],
 [6,
  7,
  17,
  18,
  22,
  25,
  26,
  28,
  33,
  37,
  39,
  41,
  42,
  49,
  51,
  56,
  61,
  62,
  63,
  64,
  66,
  73,
  75,
  78,
  79,
  82,
  83,
  85,
  86],
 [24],
 [29],
 [34],
 [36],
 [40],
 [43],
 [44],
 [45],
 [46],
 [50],
 [57, 81, 87],
 [80],
 [88]]

In [253]:
sand_mat = np.zeros([10,10])
emat = np.array(ret_list)
print(emat)

for a_ind in range(10):
    for v_ind in range(10):
        for a_node in ag_groups[a_ind]:
            for v_node in v_groups[v_ind]:
                node_indices = np.where(np.multiply(emat[:,0] == a_node, emat[:,1] == v_node))
                if len(node_indices[0]) > 0:
                    sand_mat[a_ind, v_ind] += emat[node_indices[0],2]/(len(ag_groups[a_ind])*len(v_groups[v_ind]))

[[0.00000000e+00 0.00000000e+00 2.73589916e-03]
 [0.00000000e+00 1.00000000e+00 2.51007696e-02]
 [0.00000000e+00 2.00000000e+00 2.30089365e-03]
 ...
 [8.80000000e+01 3.04000000e+02 8.67049620e-05]
 [8.80000000e+01 4.68000000e+02 5.58340176e-04]
 [8.80000000e+01 3.69000000e+02 4.05359182e-04]]


In [255]:
np.reshape(sand_mat,[1,100])

array([[1.00498578e-02, 1.40484963e-02, 2.16333611e-03, 1.56739765e-02,
        2.96954961e-02, 7.37016605e-02, 3.45579184e-03, 1.96158601e-04,
        2.13801814e-04, 3.72440851e-03, 9.35379498e-03, 2.33207403e-02,
        6.97221327e-02, 3.11481230e-04, 8.53582159e-03, 7.98999557e-03,
        8.55028759e-03, 9.36821540e-04, 2.45571816e-03, 2.13510967e-02,
        3.12300997e-02, 1.91676366e-02, 9.18712372e-02, 5.70153987e-04,
        2.65721849e-03, 1.54112512e-02, 1.28998149e-02, 1.08140936e-03,
        3.01866007e-03, 1.75843753e-02, 5.73266100e-03, 8.48645885e-03,
        7.52036624e-02, 0.00000000e+00, 2.69146251e-04, 3.19764059e-02,
        3.00969547e-02, 1.33100570e-03, 4.03763607e-04, 1.48937907e-02,
        2.12205573e-02, 1.56376831e-02, 6.22764884e-02, 6.28930413e-04,
        6.90606219e-03, 1.52505146e-02, 1.15552146e-02, 7.95113790e-04,
        2.03861722e-03, 1.46579457e-02, 3.57331686e-02, 8.13660071e-03,
        6.42728627e-02, 5.43007671e-03, 1.08030272e-02, 8.695384

In [256]:
cluster_name_strings = []
for i in range(10):
    cluster_name_strings.extend(["A_Cluster " + str(i+1)]*10)


In [257]:
cluster_name_strings

['A_Cluster 1',
 'A_Cluster 1',
 'A_Cluster 1',
 'A_Cluster 1',
 'A_Cluster 1',
 'A_Cluster 1',
 'A_Cluster 1',
 'A_Cluster 1',
 'A_Cluster 1',
 'A_Cluster 1',
 'A_Cluster 2',
 'A_Cluster 2',
 'A_Cluster 2',
 'A_Cluster 2',
 'A_Cluster 2',
 'A_Cluster 2',
 'A_Cluster 2',
 'A_Cluster 2',
 'A_Cluster 2',
 'A_Cluster 2',
 'A_Cluster 3',
 'A_Cluster 3',
 'A_Cluster 3',
 'A_Cluster 3',
 'A_Cluster 3',
 'A_Cluster 3',
 'A_Cluster 3',
 'A_Cluster 3',
 'A_Cluster 3',
 'A_Cluster 3',
 'A_Cluster 4',
 'A_Cluster 4',
 'A_Cluster 4',
 'A_Cluster 4',
 'A_Cluster 4',
 'A_Cluster 4',
 'A_Cluster 4',
 'A_Cluster 4',
 'A_Cluster 4',
 'A_Cluster 4',
 'A_Cluster 5',
 'A_Cluster 5',
 'A_Cluster 5',
 'A_Cluster 5',
 'A_Cluster 5',
 'A_Cluster 5',
 'A_Cluster 5',
 'A_Cluster 5',
 'A_Cluster 5',
 'A_Cluster 5',
 'A_Cluster 6',
 'A_Cluster 6',
 'A_Cluster 6',
 'A_Cluster 6',
 'A_Cluster 6',
 'A_Cluster 6',
 'A_Cluster 6',
 'A_Cluster 6',
 'A_Cluster 6',
 'A_Cluster 6',
 'A_Cluster 7',
 'A_Cluster 7',
 'A_Clus

In [258]:
vendor_cluster_names = []
for i in range(10):
    vendor_cluster_names.extend(["V_Cluster " + str(i+1)])
vendor_cluster_names = vendor_cluster_names*10

In [259]:
vendor_cluster_names

['V_Cluster 1',
 'V_Cluster 2',
 'V_Cluster 3',
 'V_Cluster 4',
 'V_Cluster 5',
 'V_Cluster 6',
 'V_Cluster 7',
 'V_Cluster 8',
 'V_Cluster 9',
 'V_Cluster 10',
 'V_Cluster 1',
 'V_Cluster 2',
 'V_Cluster 3',
 'V_Cluster 4',
 'V_Cluster 5',
 'V_Cluster 6',
 'V_Cluster 7',
 'V_Cluster 8',
 'V_Cluster 9',
 'V_Cluster 10',
 'V_Cluster 1',
 'V_Cluster 2',
 'V_Cluster 3',
 'V_Cluster 4',
 'V_Cluster 5',
 'V_Cluster 6',
 'V_Cluster 7',
 'V_Cluster 8',
 'V_Cluster 9',
 'V_Cluster 10',
 'V_Cluster 1',
 'V_Cluster 2',
 'V_Cluster 3',
 'V_Cluster 4',
 'V_Cluster 5',
 'V_Cluster 6',
 'V_Cluster 7',
 'V_Cluster 8',
 'V_Cluster 9',
 'V_Cluster 10',
 'V_Cluster 1',
 'V_Cluster 2',
 'V_Cluster 3',
 'V_Cluster 4',
 'V_Cluster 5',
 'V_Cluster 6',
 'V_Cluster 7',
 'V_Cluster 8',
 'V_Cluster 9',
 'V_Cluster 10',
 'V_Cluster 1',
 'V_Cluster 2',
 'V_Cluster 3',
 'V_Cluster 4',
 'V_Cluster 5',
 'V_Cluster 6',
 'V_Cluster 7',
 'V_Cluster 8',
 'V_Cluster 9',
 'V_Cluster 10',
 'V_Cluster 1',
 'V_Cluster 2',
 '